In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('responses_gemini_test.csv')

# Display the head of the dataframe
df

,encoded_related_studies,title,description,desired_criteria,messages,response
0,<STUDY>\nExample Title: A Study of the Natural...,Collection of PBMC's From Patients With Unusua...,#Study Description \nBrief Summary \nThis obse...,#Eligibility Criteria:\nInclusion Criteria - N...,<EXAMPLE_STUDIES><STUDY>\nExample Title: A Stu...,NaN
1,<STUDY>\nExample Title: An Open Registry to Me...,An Open Registry to Measure the Impact of Addi...,#Study Description \nBrief Summary \nThis regi...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN
2,"<STUDY>\nExample Title: An Open-Label, Multice...","A Phase I, Open-Label, Dose-Escalation Study o...",#Study Description \nBrief Summary \nThis is a...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN
3,<STUDY>\nExample Title: An Investigator-Initia...,Secukinumab In Patients With Moderate to Sever...,#Study Description \nBrief Summary \nThis stud...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An In...,NaN
4,<STUDY>\nExample Title: REal-world Patterns of...,Treatment Patterns And Clinical Outcomes Among...,#Study Description \nBrief Summary \nCDK4/6 in...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: REal-...,NaN
...,...,...,...,...,...,...
3988,"<STUDY>\nExample Title: A Phase 1, Open-label,...","A Phase 1, Multiple-Dose Study of the Safety a...",#Study Description \nBrief Summary \nThe purpo...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: A Pha...,NaN
3989,<STUDY>\nExample Title: The Impact of Preopera...,Do Omega-3 Fatty Acids Have Any Impact On Seru...,#Study Description \nBrief Summary \nPre- and ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: The I...,NaN
3990,<STUDY>\nExample Title: Aerobic and Resistance...,Effects of Water-based Versus Land-based Exerc...,#Study Description \nBrief Summary \nPhysical ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Aerob...,NaN
3991,<STUDY>\nExample Title: Hepassocin Levels in P...,Clusterin Level Determination and Its Associat...,#Study Description \nBrief Summary \nClusterin...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Hepas...,NaN


In [3]:
# filter the response that does not have this string
df = df[df['response'].str.contains('STEP-BY-STEP-JUSTIFICATION>')]
df

AttributeError: Can only use .str accessor with string values!

In [4]:
def gen_input(encoded_realted_studies: str, title: str, description: str):
    return f"""<RELATED_STUDIES>
{encoded_realted_studies}
</RELATED_STUDIES>

Target Study Title: {title}
Target Study Description: {description}

Task Instruction:
1. Based on the "Target Study Title" and "Target Study Description" of the target study, please create a Eligibility Criteria for the target study.
2. In <STEP-BY-STEP-JUSTIFICATION> section, please provide a detailed step-by-step logical justification for the Eligibility Criteria you created.
3. Could use example studies (in the <RELATED_STUDIES> section) if they support your justifications, but ensure the reasoning is well-explained and relevant to the study's context
4. Please provide the Eligibility Criteria in the following format (the item within the square brackets [] are the options that you can choose from):
<STEP-BY-STEP-JUSTIFICATION>
Your long step by step detailed logical justification here.
</STEP-BY-STEP-JUSTIFICATION>
<CRITERIA>
#Eligibility Criteria:
Inclusion Criteria:

* Inclusion Criteria 1
* Inclusion Criteria 2
* Inclusion Criteria 3
* ...

Exclusion Criteria:

* Exclusion Criteria 1
* Exclusion Criteria 2
* Exclusion Criteria 3
* ...

##Sex :
[MALE|FEMALE|ALL]
##Ages : 
- Minimum Age : ... Years
- Maximum Age : ... Years
- Age Group (Child: birth-17, Adult: 18-64, Older Adult: 65+) : [ADULT|CHILD|OLDER ADULT] comma separated

##Accepts Healthy Volunteers:
[YES|NO]
</CRITERIA>
"""

def gen_output(justification: str, criteria: str):
    return f"""{justification}
<CRITERIA>
{criteria.replace('INCLUSION CRITERIA', 'Inclusion Criteria').replace('EXCLUSION CRITERIA', 'Exclusion Criteria')}
</CRITERIA>
"""
import json_repair
def get_response_text(gemini_response:str):
    return json_repair.loads(gemini_response)['candidates'][0]['content']['parts'][0]['text']

In [5]:
# apply the function to the dataframe

df['input'] = df.apply(lambda x: gen_input(x['encoded_related_studies'], x['title'], x['description']), axis=1)
# df['output'] = df.apply(lambda x: gen_output(get_response_text(x['response']), x['desired_criteria']), axis=1)
# empty output
df["output"] = ""
df

,encoded_related_studies,title,description,desired_criteria,messages,response,input,output
0,<STUDY>\nExample Title: A Study of the Natural...,Collection of PBMC's From Patients With Unusua...,#Study Description \nBrief Summary \nThis obse...,#Eligibility Criteria:\nInclusion Criteria - N...,<EXAMPLE_STUDIES><STUDY>\nExample Title: A Stu...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: A S...,
1,<STUDY>\nExample Title: An Open Registry to Me...,An Open Registry to Measure the Impact of Addi...,#Study Description \nBrief Summary \nThis regi...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,
2,"<STUDY>\nExample Title: An Open-Label, Multice...","A Phase I, Open-Label, Dose-Escalation Study o...",#Study Description \nBrief Summary \nThis is a...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,
3,<STUDY>\nExample Title: An Investigator-Initia...,Secukinumab In Patients With Moderate to Sever...,#Study Description \nBrief Summary \nThis stud...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An In...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,
4,<STUDY>\nExample Title: REal-world Patterns of...,Treatment Patterns And Clinical Outcomes Among...,#Study Description \nBrief Summary \nCDK4/6 in...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: REal-...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: REa...,
...,...,...,...,...,...,...,...,...
3988,"<STUDY>\nExample Title: A Phase 1, Open-label,...","A Phase 1, Multiple-Dose Study of the Safety a...",#Study Description \nBrief Summary \nThe purpo...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: A Pha...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: A P...,
3989,<STUDY>\nExample Title: The Impact of Preopera...,Do Omega-3 Fatty Acids Have Any Impact On Seru...,#Study Description \nBrief Summary \nPre- and ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: The I...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: The...,
3990,<STUDY>\nExample Title: Aerobic and Resistance...,Effects of Water-based Versus Land-based Exerc...,#Study Description \nBrief Summary \nPhysical ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Aerob...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: Aer...,
3991,<STUDY>\nExample Title: Hepassocin Levels in P...,Clusterin Level Determination and Its Associat...,#Study Description \nBrief Summary \nClusterin...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Hepas...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: Hep...,


In [6]:
# convert the dataframe to a datasets and push to the huggingface hub
# from datasets import Dataset
# dataset = Dataset.from_pandas(df)
# # filter out the row that input is higher than 45000
# # Convert Dataset back to pandas DataFrame for filtering
# df_filtered = dataset.to_pandas()
# df_filtered = df_filtered[df_filtered['input'].str.len() < 45000]

# # Convert back to Dataset
# dataset = Dataset.from_pandas(df_filtered)
# # print len left
# display(df_filtered)
# print(len(dataset))

# dataset.push_to_hub('gemini_CoT_for_training_dataset')

# save the dataset to pickle
df.to_pickle('gemini_CoT_for_test_dataset.pkl')